# Importing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split
from PIL import Image
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
import face_recognition
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D,Activation,Flatten,Conv1D,MaxPooling1D
from keras.utils import np_utils
# to calculate accuracy
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import accuracy_score
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
import dlib
import face_recognition

# Make X data using face_encoder

In [ ]:
base_dir = os.getcwd()
folders = os.listdir(base_dir+'/data/')

categories = len(folders)

arrx = np.zeros((1,128))
arry = np.zeros((1,))
names = []
count = 0
cate = 0
for folder in folders:
    images = os.listdir(base_dir+'/data/'+folder)
    images.sort()
    names.append(folder)
    cate = cate + 1
    for image in images:
        im = cv2.imread(base_dir+'/data/'+folder+'/'+image)
        encoding = np.array(face_recognition.face_encodings(im, known_face_locations=[(0, 150, 150, 0)])[0])
        if count == 0:
            arrx[0,:] = encoding
            arry[0,] = np.array(cate)
            count = count + 1
        else:
            arrx = np.concatenate((arrx,encoding.reshape((1,128))), axis = 0)
            arry = np.concatenate((arry,np.array(cate).reshape((1,))), axis = 0)

In [ ]:
arrx.shape

In [ ]:
print(names)

# label_encoder

In [ ]:
lb = LabelEncoder()
Ydata = to_categorical(lb.fit_transform(arry))

In [ ]:
xdata, xtest, ydata, ytest = train_test_split(arrx, Ydata, test_size=0.25)

# simple model to use in face_encoder

In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=128, activation='relu'))
#model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
#model.add(Dense(16, activation='relu'))
#model.add(Dense(256, activation='relu'))
model.add(Dense(categories, activation='softmax'))

opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(xdata, ydata, epochs=300, batch_size=16, validation_data=(xtest, ytest), verbose=1)

In [ ]:
model.save('best_fit3')

In [ ]:
test_arrx = np.zeros((1,128))
im = cv2.imread('test1.jpg')
encoding = np.array(face_recognition.face_encodings(im)[0])
test_arrx[0,:] = encoding

predictions = model.predict(test_arrx,batch_size=128)

classes = np.argmax(predictions, axis = 1)

names[classes[0]]